## Scaling Notes

Integrating Romain's scaling proceedure with my workflow seems sensible. 

I would like to retain the ability to:

* Store paramters in edicts (i like writing ndp.depth, etc.)
* overwrite params with command line args
* Save and load dimensional dicts. 

Luckilty, it appears we can pickle values with dimensions. Hence, our proceedure may look like this:

* set up model with a standard ste of params in a dict (dp), only now these will have unit attched.
    * check for saved dict if starting from checkpoint, units should load automatically
    * check for commad line args and modify the magnitide (not the unit) of anu provided values
* set up a standard system of scaling (thes can live in subduction_urils) (so the sf dictonary will dissapear)
* the dimensionles param dict can now be created automatically.
 

## Tests

In [4]:
#If run through Docker we'll point at the local 'unsupported dir.'
#On hpc, the path should also include a directory holding the unsupported_dan.
import os
import sys

#this does't actually need to be protected. More a reminder it's an interim measure
try:
    sys.path.append('../../unsupported')
except:
    pass

In [9]:
import unsupported_dan.geodynamics.scaling as sca;
#from unsupported_dan.UWsubduction.base_params import *
#from unsupported_dan.UWsubduction.subduction_utils import *

In [10]:
#import unsupported.scaling as sca;
#import unsupported_dan.scaling as sca;
from easydict import EasyDict as edict
import pickle

In [11]:
u = sca.UnitRegistry

In [12]:
#Story params in a dictionaty, as per usual.
#But now with units attached

dp = edict()

dp.boxHeight = 1000. * u.kilometer
dp.tempMin   =  273.15 * u.degK

In [13]:
isinstance(dp.boxHeight, u.Quantity)
#boxLength = 2000. * u.kilometer
#isinstance(boxLength, u.Quantity)


True

In [14]:
type(testDP.boxHeight)

NameError: name 'testDP' is not defined

## Save / load dimesional edicts 

* This kind of works, but gets confused about the registries, which makes Pint throw and error

In [15]:
with open('./test_dict', 'wb') as fp:
        pickle.dump( dp , fp)

In [59]:
with open('test_dict', 'rb') as fp:
    testDP =  pickle.load(fp)

In [60]:
print(type(testDP))
print(testDP.boxHeight)
print(testDP.boxHeight.dimensionality)

#So far so good, but I find out that the loaded dictionary (of pint quantities) is now incopaatibale with the unite registry
#there is a workaround (https://github.com/hgrecco/pint/issues/146)
   #testDP.boxHeight._REGISTRY = u

#At a pinch, this could be provided as part of the checkpointing loading method

<class 'easydict.EasyDict'>
1000.0 kilometer
[length]


In [65]:
#the workaround

for key in testDP.keys():
    testDP[key]._REGISTRY = u



In [66]:
#Now all is happy

testDP.boxHeight *= 2*u.km
testDP.boxHeight

<Quantity(4000.0, 'kilometer ** 3')>

## Modify magnitudes not units

In [25]:
#We can access magnitude throug the private method, retaining the units
#testDP.boxHeight._magnitude = 10.


In [26]:
print(testDP.boxHeight)

1000.0 kilometer


## Change default scaling

By default the scaling module gives you a set of scaling coefficients to normalize your values. It can be accessed as follow:

```
sca.scaling
```

In [27]:
lengthScale = 1000*u.km

In [28]:
sca.scaling["[length]"]      = lengthScale.to_base_units()
sca.scaling

TransformedDict({'[temperature]': <Quantity(1.0, 'kelvin')>, '[time]': <Quantity(1.0, 'second')>, '[length]': <Quantity(1000000.0, 'meter')>, '[mass]': <Quantity(1.0, 'kilogram')>, '[substance]': <Quantity(1.0, 'mole')>})

In [29]:
print(testDP.boxHeight)

isinstance(testDP.boxHeight, u.Quantity)


1000.0 kilometer


False

In [33]:
#testDP.boxHeight * u.km

In [30]:
test = sca.nonDimensionalize(testDP.boxHeight)
test 

ValueError: Cannot operate with Quantity and Quantity of different registries.

In [47]:
type(testDP.boxHeight)

pint.unit.Quantity

In [48]:
type(boxLength)

pint.unit.Quantity

In [44]:
boxLength = 2000. * u.kilometer
sca.nonDimensionalize(boxLength)

2.0

## Non-Dimensionalize the dictionary

<Quantity(10000.0, 'meter')>